In [14]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -U pip
    !pip install dspy-ai
    !pip install openai~=0.28.1
    # !pip install -e $repo_path

import dspy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Already up to date.


AttributeError: module 'pyarrow.lib' has no attribute 'ListViewType'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load a Dataset

In [ ]:
url= "/content/drive/MyDrive/output/rest15/test.tsv"
import pandas as pd

# Step 3: 读取TSV文件
file_path = url
df = pd.read_csv(file_path, sep='\t')
df_first_50 = df.head(50)

# Assuming the original DataFrame is named df
# Extracting the 'review' and 'aspect' columns
reviews = df_first_50['review'].tolist()
aspects = df_first_50['aspect'].tolist()
sentiments = df_first_50['sentiment'].tolist()

# Combining the extracted columns into the desired format
dev = [(review, aspect, "neutral" if sentiment == 0 else ("positive" if sentiment == 1 else "negative")) for review, aspect, sentiment in zip(reviews, aspects, sentiments)]

# Converting to the desired format using dspy.Example
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

# Print the result to verify
for example in dev:
    print(example)


NameError: name 'dspy' is not defined

Define a **Dataset**

In [ ]:
train = [("sarah palin is awesome. | yes she is", "sarah palin", "positive"),
      ("russell brand and katy perry suit each other. they 're both weird.","katy perry", "negative"),
      ("lindsay lohan must hate herself : samantha ronson has more followers on twitter. you know you 've hit rock bottom when ...", "lindsay lohan", 'neutral'),
      ('and he has a vision too ! mirzan mahatir is looking like steve jobs right now.', 'steve jobs','neutral'),
      ("i love google ! as a whole with all its services","google","positive")
      ]

#
train = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in train]

dev = [("my 3-year-old was amazed yesterday to find that ' real ' 10 pin bowling is nothing like it is on the wii ...", "wii", "neutral"),
    ("listening 2 '' 3 '' by ms britney spears cant wait 4 the video on oct . 19 you guys need 2 love her new single also buy her single collection nov . 24 !", "britney spears", "positive"),
    ("ff markieai likes animals . hates spiders . hates charlie sheen . likes cookies .", "charlie sheen", "negative"),
    ("espn soccer net news ardiles backs maradona for world cup glory", "maradona", "positive"),
    ("-lsb- happy birthday git * git ! -rsb- ily ! ... some reason im very tired ! but today was really fun ! :-rrb- ugh my ipod died !", "ipod", "neutral"),
    ("hilary swank as amelia earhart is actually pretty great casting.", "hilary swank", "positive"),
    ("video : serena williams gets x-rated for new tennis game ad ! .", "serena williams", "negative"),
    ("lmao i know !! i dnt like harry potter he 's so english !! hehehe its raining here too smdh", "harry potter", "negative"),]
#
dev = [dspy.Example(sentence=sentence, aspect=aspect, sentiment=sentiment).with_inputs('sentence', 'aspect') for sentence, aspect, sentiment in dev]

Define a **Metric**

In [ ]:
def sentiment_classification_metric(example, pred, trace=None):
    return example.sentiment.lower() == pred.sentiment.lower()

Create the DSPy Constrain & Analogies & Intro **Context Program**

In [ ]:
import dspy
import openai
from dspy import OpenAI

openai.api_base = "https://api.gpts.vin/v1"
openai.api_key = "sk-FhKeNgB7ZgqvpZqsF357D67Eaa8c42339b02Cf57C56a7d19"

lm = OpenAI(api_base=openai.api_base,
             api_key=openai.api_key,
             model='gpt-3.5-turbo')

dspy.settings.configure(lm=lm)

# Define the DSPy implementation of sentiment polarity analysis with an aspect
class AspectBasedSentimentAnalysis(dspy.Signature):
    """Analyze the sentiment polarity of a given sentence and its aspect."""
    context = dspy.InputField()
    sentence = dspy.InputField(desc="The sentence to analyze.")
    aspect = dspy.InputField()
    sentiment = dspy.OutputField(desc="positive, neutral, or negative.")
class AspectBasedSentimentAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.analyze_sentiment = dspy.ChainOfThought(AspectBasedSentimentAnalysis)
    def forward(self, sentence, aspect):
        # Use GPT to create context introduction for the aspect
        context = self.get_context(sentence, aspect)
        print(f"Context:\n", context)
        prediction = self.analyze_sentiment(sentence=sentence, aspect=aspect, context=context)
        print(f"Prediction:\n", prediction)
        self.write_to_file(aspect, context, sentence, prediction)
        return dspy.Prediction(context=context, sentiment=prediction.sentiment)
    def get_constrain_context(self, sentence, aspect):
        prompt = (
            f"# Your task is to answer the following questions with 'Yes' or 'No' and input the answer into the slot $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is positive? $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is negative? $PRED$\n"
            f"# From '{sentence}', can we infer that the {aspect} is neutral? $PRED$\n"
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=120,
            n=1,
            stop=None
        )
        constrain_context = response.choices[0].message['content'].strip()
        return constrain_context
    def get_intro_context(self, aspect):
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Please breifly introduce '{aspect}'."}
            ],
            max_tokens=50,
            n=1,
            stop=None
        )

        intro_context = response.choices[0].message['content'].strip()
        return intro_context
    def get_analogies_context(self, aspect):
        prompt = (
            f"Recall three Aspect-Based Sentiment Analysis examples with the aspect '{aspect}' included in the sentence. Each example should include:\n1. The sentence to be analyzed.\n2. The predicted sentiment polarity of the aspect ('positive', 'neutral', 'negative')."
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=120,
            n=1,
            stop=None
        )
        analogies_context = response.choices[0].message['content'].strip()
        return analogies_context
    def get_context(self, sentence, aspect):
        constrain_context = self.get_constrain_context(sentence, aspect)
        intro_context = self.get_intro_context(aspect)
        analogies_context = self.get_analogies_context(aspect)
        return f"# Constrain context:{constrain_context}\n\n# Background context:{intro_context}\n\n# Analogies context:{analogies_context}"
    def analyze_sentiment(self, sentence, aspect, context):
          prompt = (
                f"# Aspect: {aspect}\n"
                f"# Context: {context}\n"
                f"# Sentence: {sentence}\n"
                f"# Please analyze the sentiment polarity (positive, neutral, negative) of '{aspect}' in the sentence:\n"
            )
          response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=50,
                n=1,
                stop=None
            )

          sentiment = response.choices[0].message['content'].strip()
          return sentiment
    def write_to_file(self, aspect, context, sentence, prediction):
        with open('output.txt', 'a') as f:
            f.write(f"Aspect: {aspect}\n")
            f.write(f"Sentence: {sentence}\n")
            f.write(f"Context: {context}\n")
            f.write("----------------------------------------------------\n")
            f.write(f"Rationale: {prediction.rationale}\n")
            f.write(f"Sentiment: {prediction.sentiment}\n")
            f.write("++++++++++++++++++++++++++++++++++++++++++++++++++++\n")

In [ ]:
# analyzer = AspectBasedSentimentAnalyzer()

# examples = [
#     {'sentence': 'sarah palin is awesome.', 'aspect': 'sarah palin'},
#     {'sentence': 'The service at the restaurant was terrible.', 'aspect': 'service'},
#     {'sentence': 'I love the camera on this phone.', 'aspect': 'camera'},
#     {'sentence': 'The battery life of this laptop is too short.', 'aspect': 'battery life'}
# ]

# with open('sentiment_analysis_results1.txt', 'w') as file:
#     for example in examples:
#         result = analyzer(example['sentence'], example['aspect'])
#         file.write(f"Sentence: {example['sentence']}\n")
#         file.write(f"Aspect: {example['aspect']}\n")
#         file.write(f"Context: {result.context}\n")
#         file.write(f"Sentiment: {result.sentiment}\n")
#         file.write("\n")

In [ ]:
# Compile the Program
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)
teleprompter = BootstrapFewShot(metric=sentiment_classification_metric, **config)

# Compile the program
optimized_classifier = teleprompter.compile(AspectBasedSentimentAnalyzer(), trainset=train)

  0%|          | 0/5 [00:00<?, ?it/s]

Context:
 # Constrain context:From 'sarah palin is awesome. | yes she is', we can infer that the sarah palin is positive. Yes

# Background context:Sarah Palin is an American politician, commentator, author, and reality television personality. She served as the 9th governor of Alaska from 2006 to 2009 and was the Republican Party nominee for Vice President in the 2008 presidential election

# Analogies context:Certainly! Here are three Aspect-Based Sentiment Analysis examples with the aspect 'Sarah Palin' included:

1. Sentence: "Sarah Palin delivered a charismatic speech that energized the crowd."
   Predicted sentiment polarity: Positive

2. Sentence: "Many people are divided on Sarah Palin's political views."
   Predicted sentiment polarity: Neutral

3. Sentence: "Critics often criticize Sarah Palin for her controversial statements."
   Predicted sentiment polarity: Negative


 20%|██        | 1/5 [00:07<00:28,  7.07s/it]

Prediction:
 Prediction(
    rationale='produce the sentiment. We know from the background context that Sarah Palin is a well-known politician and public figure. The positive sentiment is likely due to her charisma, energy, and appeal to her supporters.',
    sentiment='positive'
)
Context:
 # Constrain context:From 'russell brand and katy perry suit each other. they're both weird.', we can infer that the katy perry is neutral.

# Background context:Katy Perry is an American singer, songwriter, and television judge. She gained worldwide fame for her catchy pop songs and energetic performances. With hit singles such as "I Kissed a Girl," "Firework," and "Roar," Katy Perry

# Analogies context:


 40%|████      | 2/5 [00:22<00:36, 12.22s/it]

...

Prediction:
 Prediction(
    rationale='produce the sentiment. The statement "lindsay lohan must hate herself" implies a negative sentiment towards Lindsay Lohan. Additionally, the comparison to Samantha Ronson having more followers on Twitter suggests a feeling of inferiority, which also adds to the negative sentiment.',
    sentiment='negative'
)
Context:
 # Constrain context:From the given statement 'and he has a vision too! mirzan mahatir is looking like Steve Jobs right now.', we cannot infer whether the perception of Steve Jobs is positive, negative, or neutral.

# Background context:Steve Jobs was an American entrepreneur, business magnate, and co-founder of Apple Inc. He is widely regarded as a pioneer of the personal computer revolution, and was known for his innovative vision and design aesthetic. Jobs played a key role in the development

# Analogies context:Sure! Here are three aspect-based sentiment analysis examples with the aspect "Steve Jobs" included:

1. Sentence: "St

 80%|████████  | 4/5 [01:01<00:16, 16.59s/it]

Prediction:
 Prediction(
    rationale='analyze the sentiment polarity of the sentence and its aspect.\n\nSentence: i love google ! as a whole with all its services\n\nAspect: google\n\nReasoning: Let\'s think step by step in order to determine the sentiment. The sentence expresses admiration and affection towards Google, emphasizing appreciation for its entirety and services. The use of exclamation mark and the phrase "love Google" indicate a strongly positive sentiment towards the company.',
    sentiment='positive\n\n---\n\nSentence: sarah palin is awesome. | yes she is\n\nAspect: sarah palin\n\nReasoning: Let\'s think step by step in order to determine the sentiment. Both sentences affirm a positive opinion of Sarah Palin. The use of "awesome" directly expresses admiration and positivity towards her, reinforced by the agreement in the following sentence ("yes she is"). This confirms a consistent positive sentiment towards Sarah Palin.\n\nSentiment: positive\n\n---\n\nSentence: russ

100%|██████████| 5/5 [01:23<00:00, 16.73s/it]

Prediction:
 Prediction(
    rationale='produce the sentiment. We can infer that the sentiment towards Google in this aspect is positive because the sentence expresses love and admiration for Google and its services as a whole.',
    sentiment='positive'
)


Evaluate the Program

In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator
evaluate = Evaluate(devset=dev, metric=sentiment_classification_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate the optimized classifier
evaluate(optimized_classifier)

  0%|          | 0/8 [00:00<?, ?it/s]

Context:
 # Constrain context:# From 'espn soccer net news ardiles backs maradona for world cup glory', can we infer that the maradona is positive? No
# From 'espn soccer net news ardiles backs maradona for world cup glory', can we infer that the maradona is negative? No
# From 'espn soccer net news ardiles backs maradona for world cup glory', can we infer that the maradona is neutral? Yes

# Background context:Maradona was an Argentine professional soccer player widely regarded as one of the greatest of all time. He was known for his incredible skill, speed, and charisma on the field, and he led Argentina to victory in the 1986 World Cup.

# Analogies context:Sure, here are three example sentences with the aspect "maradona" and the predicted sentiment polarity:

1. "Maradona was a legendary footballer and his impact on the game will always be remembered."
   - Predicted sentiment polarity: positive

2. "The career of Maradona was marred by controversy and scandals off the pitch."
   -

Average Metric: 0 / 1  (0.0):  12%|█▎        | 1/8 [00:09<01:07,  9.59s/it]

Prediction:
 Prediction(
    rationale='produce the sentiment. From the background context, we know that Maradona is widely regarded as one of the greatest soccer players of all time. The aspect of Maradona in this sentence is being backed for World Cup glory, which indicates a neutral sentiment as it does not express explicit positivity or negativity towards Maradona.',
    sentiment='neutral'
)


Average Metric: 0 / 2  (0.0):  25%|██▌       | 2/8 [00:10<00:25,  4.24s/it]

...

Average Metric: 4 / 7  (57.1):  88%|████████▊ | 7/8 [00:26<00:02,  2.82s/it]

Prediction:
 Prediction(
    rationale='produce the sentiment. The sentiment towards Harry Potter in this aspect is negative. The speaker expresses dislike towards Harry Potter, mentioning that they don\'t like him because he\'s "so English." This indicates a negative sentiment towards the character or series.',
    sentiment='negative'
)
Context:
 # Constrain context:From the given video title, it is not possible for me to determine Serena Williams' sentiment based solely on "serena williams gets x-rated for new tennis game ad!" Therefore, I cannot infer whether Serena Williams is positive, negative, or neutral from the given information.

# Background context:Serena Williams is a professional tennis player from the United States and is considered one of the greatest players in the history of the sport. With numerous Grand Slam titles and Olympic gold medals to her name, she has made a significant impact on women's tennis

# Analogies context:Certainly! Here are three Aspect-Based Sen

Average Metric: 4 / 8  (50.0): 100%|██████████| 8/8 [01:09<00:00,  8.68s/it]

Prediction:
 Prediction(
    rationale='produce the sentiment. The sentence provided is a video title, "serena williams gets x-rated for new tennis game ad!". This title does not directly convey sentiment towards Serena Williams herself but rather describes the content of a promotional video involving her.\n\nAspect: serena williams\n\nReasoning: The sentiment analysis cannot be reliably determined from this sentence alone, as it focuses on the content of a promotional video rather than Serena Williams\' personal attributes or actions. The phrase "x-rated" typically denotes content intended for adults and does not provide insight into Serena Williams\' performance, achievements, or public perception.',
    sentiment='cannot be inferred'
)


50.0